---
title: "2023_Kronensicherung_Plesse_004_Combination_and_Cleaning"
author: "Kyell Jensen"
date: "2024-08-06"
format: pdf
editor: visual
---

# 2023_Kronensicherung_Plesse_Combination_and_Cleaning

## Kombinieren und Bereinigen der Daten von LineScale3, TreeQinetic und Versuchsaufzeichung

Nutze eine geeignete Python 3.11 Umgebung (z. B. virtuelle Environment).

## Arbeitsumgebung vorbereiten

### IMPORT: Importieren von Standardbibliotheken

Die folgenden Bibliotheken werden importiert, um grundlegende Funktionen für Strukturierung, Datenverarbeitung, Plotting und statistische Auswertung bereit zu stellen.

In [31]:
# Struktur
from pathlib import Path
from typing import Dict, List

# Datenverarbeitung
import json
from IPython.display import Markdown, display
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

### IMPORT: Importiere eigene Packete

Lade allgemeine Export-Funktionen, um die Daten als Latex-Tabellen zu exportieren

In [32]:
from kj_core.utils.latex_export import (
    save_latex_table,
    build_data_dict_df
)

## IMPORT: Daten Import

Lege Pfade für Daten-Importe, Daten-Exporte etc. fest (ggf. anpassen an eigene Verzeichnisstruktur), ausgelagert in gemeinsame Config für verschiedene Notebooks

In [33]:
# Importiere alle Einstellungen aus der project_config.py
from project_config import (
    data_path,
    data_export_directory,
    latex_export_directory
)

In [34]:
# Liste aller Dateinamen
filenames = [
    "tree.feather",
    "sensor.feather",
    "sensor_data_dict.json",
    "series.feather",
    "series_data_dict.json",
    "ls3_data_dict.json",
    "ls3.feather",
    "ptq_data_dict.json",
    "ptq.feather",
]

In [35]:
def load_file(path: Path):
    try:
        if path.suffix == ".feather":
            df_now = pd.read_feather(path)
            print(f"[Info] Feather geladen: {path.name}")
            return "df", path.stem, df_now
        elif path.suffix == ".json":
            with open(path, "r", encoding="utf-8") as f:
                data_dict_now = json.load(f)
                print(f"[Info] JSON geladen: {path.name}")
                return "dict", path.stem, data_dict_now
        else:
            print(f"[Warnung] Unbekannter Dateityp übersprungen: {path.name}")
            return None
    except Exception as e:
        print(f"[Fehler] Laden fehlgeschlagen für '{path.name}': {e}")
        return None


In [36]:
# Container
dfs = {}
data_dicts = {}

for file in filenames:
    result = load_file(data_export_directory / file)
    if result:
        kind, name_stem, content = result
        if kind == "df":
            dfs[f"{name_stem}_df"] = content
        elif kind == "dict":
            data_dicts[f"{name_stem}"] = content

[Info] Feather geladen: tree.feather
[Info] Feather geladen: sensor.feather
[Info] JSON geladen: sensor_data_dict.json
[Info] Feather geladen: series.feather
[Info] JSON geladen: series_data_dict.json
[Info] JSON geladen: ls3_data_dict.json
[Info] Feather geladen: ls3.feather
[Info] JSON geladen: ptq_data_dict.json
[Info] Feather geladen: ptq.feather


In [37]:
dfs.keys()

dict_keys(['tree_df', 'sensor_df', 'series_df', 'ls3_df', 'ptq_df'])

In [38]:
data_dicts.keys()

dict_keys(['sensor_data_dict', 'series_data_dict', 'ls3_data_dict', 'ptq_data_dict'])

In [39]:
# IDE-Hints: Feather-Dateien (DataFrames)
tree_df: pd.DataFrame = dfs.get("tree_df")
sensor_df: pd.DataFrame = dfs.get("sensor_df")
series_df: pd.DataFrame = dfs.get("series_df")
ls3_df: pd.DataFrame = dfs.get("ls3_df")
ptq_df: pd.DataFrame = dfs.get("ptq_df")

# IDE-Hints: Data Dictionary Dateien (JSON → dict)
sensor_data_dict: dict = data_dicts.get("sensor_data_dict")
series_data_dict: dict = data_dicts.get("series_data_dict")
ls3_data_dict: dict = data_dicts.get("ls3_data_dict")
ptq_data_dict: dict = data_dicts.get("ptq_data_dict")


In [40]:
ptq_df

,id,file_name,sensor_name,sample_rate,max_strain,max_compression,max_strain_osc,max_compression_osc,m_amplitude,m_amplitude_2,...,damping_ratio,pearson_r,p_value,r2,mse,rmse,nrmse,cv,mae,nmae
0,1,PTQ_Meas_100346.txt,Elasto(90),4.003157,368.0,-168.1,363.5,-168.1,265.80,162.10,...,0.365278,0.956182,0.000000e+00,0.877885,594.489599,24.382157,0.045868,-60.280300,10.359435,0.019488
1,1,PTQ_Meas_100346.txt,Elasto(92),3.941160,240.9,-86.6,239.9,-86.6,163.25,76.00,...,0.643421,0.931204,0.000000e+00,0.838016,177.244360,13.313315,0.040776,-1.212033,6.809497,0.020856
2,1,PTQ_Meas_100346.txt,Elasto(95),4.003157,215.0,-119.1,190.2,-119.1,154.65,82.25,...,0.647691,0.932709,0.000000e+00,0.866563,220.603098,14.852713,0.048022,-0.520646,8.316242,0.026888
3,1,PTQ_Meas_100346.txt,Elasto(98),3.941160,192.8,-132.8,141.6,-132.8,137.20,92.50,...,0.540556,0.979671,0.000000e+00,0.955699,67.452450,8.212944,0.029932,-0.399388,5.589269,0.020370
4,2,PTQ_Meas_101814.txt,Elasto(90),3.447691,429.6,-202.2,417.5,-202.2,309.85,162.95,...,0.503994,0.943158,0.000000e+00,0.844431,580.017090,24.083544,0.038866,-3.478901,11.160700,0.018011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,28,PTQ_Meas_155805.txt,Elasto(98),2.391119,149.1,-108.4,120.3,-108.4,114.35,95.15,...,0.820843,0.952398,0.000000e+00,0.903178,94.517863,9.722030,0.042510,-1.489630,6.965469,0.030457
112,29,PTQ_Meas_160907.txt,Elasto(90),2.967190,387.9,-94.4,383.8,-94.4,239.10,107.30,...,0.475166,0.805759,4.900737e-201,0.621876,1332.144003,36.498548,0.076325,12.131841,16.258373,0.033999
113,29,PTQ_Meas_160907.txt,Elasto(92),2.967190,263.4,-28.3,259.7,-22.0,140.85,39.30,...,0.994797,0.824291,3.874928e-218,0.634750,488.029395,22.091387,0.078422,5.346908,10.226893,0.036304
114,29,PTQ_Meas_160907.txt,Elasto(95),2.967190,237.6,-23.9,228.1,-23.9,126.00,46.20,...,0.777837,0.851855,1.085132e-247,0.561725,410.272163,20.255176,0.080378,3.804639,9.617778,0.038166


## MERGE: Zusammenführen der Daten von LS3, PTQ und Versuchsprotokoll

Beginne mit PTQ-Daten, füge dann schrittweise die Daten des Versuchsablaufes, der Sensorpositionierung und Kraftmessung hinzu

In [41]:
df = pd.merge(ptq_df, series_df, on='id', how='left')
# df

Ergänze die Daten der Sensorpositionierung

In [42]:
# Merging df and elasto_df
# Geräte vom Typ Elastometer auswählen aus dem sensor_df
elasto_df = sensor_df[sensor_df["type"] == "Elasto"].copy()

# Passe die sensor_namen einheitlich für beide DataFrames an
elasto_df["sensor_name"] = elasto_df["type"].astype(str) + "(" + elasto_df["sensor_id"].astype(str) + ")"

# Perform the left join on sensor_name
df = df.merge(elasto_df, on="sensor_name", how="left")

# Gewünschte Reihenfolge
elasto_names = ["Elasto(90)", "Elasto(92)", "Elasto(95)", "Elasto(98)"]
# Definieren als geordneter CategoricalDtype
elasto_cat_type = CategoricalDtype(categories=elasto_names, ordered=True)
# Setzen des Typs auf die Spalte
df["sensor_name"] = df["sensor_name"].astype(elasto_cat_type)
# df

Ergänze die LS3-Daten

In [43]:
# Function to prepare ls3_metadata
def extract_ls3_subset(df, sensor_id, new_prefix):
    return (df.query(f"sensor_id == '{sensor_id}'")
            .drop('sensor_id', axis=1)
            .add_prefix(new_prefix)
            .rename(columns={f'{new_prefix}measurement_id': sensor_id}))

# Merging ls3_metadata with the main DataFrame
df = df.merge(extract_ls3_subset(ls3_df, '14:BF:E6', 'rope_'), on='14:BF:E6', how='left')     
df = df.merge(extract_ls3_subset(ls3_df, '14:99:1E', 'cable_'), on='14:99:1E', how='left')     

In [44]:
df

,id,file_name,sensor_name,sample_rate,max_strain,max_compression,max_strain_osc,max_compression_osc,m_amplitude,m_amplitude_2,...,cable_timing_correction_factor,cable_datetime_start,cable_datetime_end,cable_duration,cable_length,cable_max,cable_mean,cable_median,cable_min,cable_release
0,1,PTQ_Meas_100346.txt,Elasto(90),4.003157,368.0,-168.1,363.5,-168.1,265.80,162.10,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,PTQ_Meas_100346.txt,Elasto(92),3.941160,240.9,-86.6,239.9,-86.6,163.25,76.00,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,PTQ_Meas_100346.txt,Elasto(95),4.003157,215.0,-119.1,190.2,-119.1,154.65,82.25,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,PTQ_Meas_100346.txt,Elasto(98),3.941160,192.8,-132.8,141.6,-132.8,137.20,92.50,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,PTQ_Meas_101814.txt,Elasto(90),3.447691,429.6,-202.2,417.5,-202.2,309.85,162.95,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,28,PTQ_Meas_155805.txt,Elasto(98),2.391119,149.1,-108.4,120.3,-108.4,114.35,95.15,...,0.9,2022-03-23 17:05:21,2022-03-23 17:05:37.199296875,16.199296,23040.0,"[3855.0, 2.36]",0.320037,0.31,"[48.0, 0.0]",NaN
112,29,PTQ_Meas_160907.txt,Elasto(90),2.967190,387.9,-94.4,383.8,-94.4,239.10,107.30,...,0.9,2022-03-23 17:16:51,2022-03-23 17:17:07.199296875,16.199296,23040.0,"[3881.0, 1.74]",0.112564,0.05,"[28.0, 0.0]",NaN
113,29,PTQ_Meas_160907.txt,Elasto(92),2.967190,263.4,-28.3,259.7,-22.0,140.85,39.30,...,0.9,2022-03-23 17:16:51,2022-03-23 17:17:07.199296875,16.199296,23040.0,"[3881.0, 1.74]",0.112564,0.05,"[28.0, 0.0]",NaN
114,29,PTQ_Meas_160907.txt,Elasto(95),2.967190,237.6,-23.9,228.1,-23.9,126.00,46.20,...,0.9,2022-03-23 17:16:51,2022-03-23 17:17:07.199296875,16.199296,23040.0,"[3881.0, 1.74]",0.112564,0.05,"[28.0, 0.0]",NaN


#### Analysiere Metrik Warnungen


In [45]:
# Filtere den DataFrame auf Zeilen, bei denen metrics_warning True ist
warnings_df = df[df['metrics_warning']]

# Gruppiere nach 'treatment' und 'sensor_name', zähle Warnungen und sammle IDs
warnings_summary = (
    warnings_df
    .groupby(['treatment', 'sensor_name'], observed=False)
    .agg(
        warning_count=('metrics_warning', 'size'),
         ids=('id', lambda x: sorted(x.astype(str).tolist()))
    )
    .reset_index()
)

warnings_summary = warnings_summary[warnings_summary['warning_count'] > 0]
warnings_summary

,treatment,sensor_name,warning_count,ids
4,gefa_dynamic,Elasto(90),1,[14]
9,cobra_static,Elasto(92),3,"[21, 23, 28]"
10,cobra_static,Elasto(95),5,"[22, 24, 25, 26, 27]"


In [46]:
# LaTeX-String erstellen
latex_string = warnings_summary.to_latex(
    index=False, 
    escape=True, 
    column_format="llll",  # Spaltenformat angepasst
    float_format="{:0.2f}".format
)

caption = "Feldversuch 2 - Ergebnisse, Schwingung, Warnung Anpassungsgüte"
caption_long = "Feldversuch 2 - Ergebnisse, Schwingung, Warnung Anpassungsgüte, r < 0,75"

save_latex_table(latex_string, caption, latex_export_directory, caption_long)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_ergebnisse_schwingung_warnung_anpassungsgute.tex


### EXPORT: Ungefilterte Daten exportieren für Externe (.feather, .csv)
 

In [47]:
df.to_feather(data_export_directory / "_dataset_full.feather")
df.to_csv(data_export_directory / "_dataset_full.csv", sep=";", index=True, encoding="utf-8")

## CLEANING: Daten bereinigen und Filtern

In [48]:
# Liste der Spalten, die beibehalten werden sollen
select_cols = ['id', 'rope_datetime', 'treatment', 'release_force_target', 
               'rope_release', 'cable_max',
               'sensor_name', 'location', 'direction', 'height', 'diameter',
               'max_strain', 'max_compression',
               'm_amplitude', 'm_amplitude_2', 
               'initial_amplitude', 'damping_coeff', 'frequency_damped', 'phase_angle', 'y_shift', 'x_shift', 'frequency_undamped', 'damping_ratio', 
               'metrics_warning', 'pearson_r', 'nrmse', 'nmae'
               ]

# DataFrame filtern
df = (
    df[select_cols]
    .query("release_force_target in [2.0, 2.4, 2.8] and treatment in ['free', 'gefa_dynamic', 'cobra_static']")
    .copy()
)

In [49]:
# Nicht verwendete Kategorien entfernen
df["treatment"] = df["treatment"].cat.remove_unused_categories()

In [50]:
# cable_max enthält noch ein np.ndarray mit index etc., nur Messwerte F benötigt
df['cable_max'] = df['cable_max'].apply(
    lambda x: float(x[1]) if isinstance(x, (tuple, list, np.ndarray)) and len(x) > 1 else x
)

In [51]:
df.head(10)

,id,rope_datetime,treatment,release_force_target,rope_release,cable_max,sensor_name,location,direction,height,...,frequency_damped,phase_angle,y_shift,x_shift,frequency_undamped,damping_ratio,metrics_warning,pearson_r,nrmse,nmae
4,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,Elasto(90),StB,elongation,16.55,...,0.441518,0.200000,-5.287164,0.116641,0.442936,0.503994,False,0.943158,0.038866,0.018011
5,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,Elasto(92),StB,elongation,11.6,...,0.432441,-0.200000,-14.233644,-0.009730,0.436601,0.873595,False,0.929554,0.034560,0.016183
6,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,Elasto(95),StA,elongation,11.6,...,0.422702,-0.200000,-19.011309,-0.123043,0.426319,0.823807,False,0.908058,0.044873,0.024274
7,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,Elasto(98),StA,elongation,16.85,...,0.443529,-0.065967,-1.099515,0.118608,0.445993,0.663224,False,0.980663,0.029621,0.021087
8,3,2022-03-23 11:31:12,free,2.8,2.76205,NaN,Elasto(90),StB,elongation,16.55,...,0.440653,0.200000,7.775284,0.097887,0.441317,0.344951,False,0.957380,0.043035,0.018619
9,3,2022-03-23 11:31:12,free,2.8,2.76205,NaN,Elasto(92),StB,elongation,11.6,...,0.439321,-0.075771,7.052210,0.094719,0.441380,0.608931,False,0.933463,0.043534,0.021255
10,3,2022-03-23 11:31:12,free,2.8,2.76205,NaN,Elasto(95),StA,elongation,11.6,...,0.436023,-0.200000,-3.785755,-0.075741,0.438127,0.618021,False,0.938307,0.039020,0.022498
11,3,2022-03-23 11:31:12,free,2.8,2.76205,NaN,Elasto(98),StA,elongation,16.85,...,0.441350,-0.200000,-17.747289,-0.203345,0.442888,0.525070,False,0.974139,0.028693,0.020715
12,4,2022-03-23 11:54:33,free,2.8,2.73950,NaN,Elasto(90),StB,elongation,16.55,...,0.439065,0.200000,-23.445975,0.120929,0.439764,0.354677,False,0.945320,0.049764,0.021509
13,4,2022-03-23 11:54:33,free,2.8,2.73950,NaN,Elasto(92),StB,elongation,11.6,...,0.436850,-0.200000,-19.454720,0.061912,0.438829,0.598701,False,0.922665,0.050822,0.020659


### EDIT: Datendokumentation an zusammengeführte und gefilterte Daten anpassen

Passe Datendokumentation den zusammengeführten und gefilterten Daten an, ergänze weitere später berechnete Features, erstellt vollstände aktuelle Datendokumentation

In [52]:
# Lade das Dictionary mit der Daten Dokumentation
with open(data_path / "calc_strain_data_dict.json", "r", encoding="utf-8") as f:
    calc_strain_data_dict = json.load(f)
    
select_cols += list(calc_strain_data_dict.keys())

In [53]:
# Rope-Keys erzeugen
ls3_rope_data_dict = {f"rope_{k}": v for k, v in ls3_data_dict.items()}
# Cable-Keys erzeugen
ls3_cable_data_dict = {    f"cable_{k}": v for k, v in ls3_data_dict.items()}

In [54]:
# Alles zusammenführen
full_data_dict = {
    **sensor_data_dict,
    **series_data_dict,
    **ls3_rope_data_dict,
    **ls3_cable_data_dict,
    **ptq_data_dict,
    **calc_strain_data_dict
}

In [55]:
# 1. sensor → Deutsch
if "sensor" in full_data_dict:
    full_data_dict["sensor"]["Deutsch"] = "Elastometer"

# 2. cable_max → Beschreibung, Deutsch, Zeichen
if "cable_max" in full_data_dict:
    full_data_dict["cable_max"]["Beschreibung"] = "Maximale gemessene Kraftspitze in der KS"
    full_data_dict["cable_max"]["Deutsch"] = "Kraftspitze KS"
    full_data_dict["cable_max"]["Zeichen"] = "$F_{\\mathrm{cable, max}}$"

In [56]:
data_dict = {key: full_data_dict[key] for key in select_cols if key in full_data_dict}
data_dict

{'id': {'Kategorie': 'ptq',
  'Zeichen': 'ID',
  'Deutsch': 'ID Messung',
  'Datentyp': 'int64',
  'Einheit': '-',
  'Beschreibung': 'Eindeutige ID der Messung'},
 'rope_datetime': {'Kategorie': 'ls3',
  'Zeichen': '$t$',
  'Deutsch': 'Zeitstempel',
  'Datentyp': 'object',
  'Einheit': '-',
  'Beschreibung': 'Startzeitpunkt der Messung laut Gerät'},
 'treatment': {'Kategorie': 'series',
  'Zeichen': 'treatment',
  'Deutsch': 'Behandlung',
  'Datentyp': 'category',
  'Einheit': '-',
  'Beschreibung': 'Art der KS: \\texttt{free}, \\texttt{gefa\\_dynamic}, \\texttt{cobra\\_static}'},
 'release_force_target': {'Kategorie': 'series',
  'Zeichen': '$F_{\\mathrm{release,target}}$',
  'Deutsch': 'Vorspannkraft-Soll',
  'Datentyp': 'float64',
  'Einheit': 'kN',
  'Beschreibung': 'Geplante Vorspannkraft im Zugseil bei Release'},
 'rope_release': {'Kategorie': 'ls3',
  'Zeichen': '$F_{\\mathrm{release}}$',
  'Deutsch': 'Vorspannkraft-Ist',
  'Datentyp': 'float64',
  'Einheit': 'kN',
  'Beschreibu

In [57]:
df.head(100)

,id,rope_datetime,treatment,release_force_target,rope_release,cable_max,sensor_name,location,direction,height,...,frequency_damped,phase_angle,y_shift,x_shift,frequency_undamped,damping_ratio,metrics_warning,pearson_r,nrmse,nmae
4,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,Elasto(90),StB,elongation,16.55,...,0.441518,0.200000,-5.287164,0.116641,0.442936,0.503994,False,0.943158,0.038866,0.018011
5,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,Elasto(92),StB,elongation,11.6,...,0.432441,-0.200000,-14.233644,-0.009730,0.436601,0.873595,False,0.929554,0.034560,0.016183
6,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,Elasto(95),StA,elongation,11.6,...,0.422702,-0.200000,-19.011309,-0.123043,0.426319,0.823807,False,0.908058,0.044873,0.024274
7,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,Elasto(98),StA,elongation,16.85,...,0.443529,-0.065967,-1.099515,0.118608,0.445993,0.663224,False,0.980663,0.029621,0.021087
8,3,2022-03-23 11:31:12,free,2.8,2.76205,NaN,Elasto(90),StB,elongation,16.55,...,0.440653,0.200000,7.775284,0.097887,0.441317,0.344951,False,0.957380,0.043035,0.018619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,25,2022-03-23 16:44:31,cobra_static,2.4,2.45465,2.80,Elasto(98),StA,elongation,16.85,...,0.526233,0.200000,0.004065,-0.123942,0.530764,0.826263,False,0.939117,0.040525,0.028335
100,26,2022-03-23 16:56:04,cobra_static,2.0,1.99840,2.33,Elasto(90),StB,elongation,16.55,...,0.543590,-0.200000,-5.140277,0.088797,0.546791,0.682871,False,0.890898,0.044373,0.022288
101,26,2022-03-23 16:56:04,cobra_static,2.0,1.99840,2.33,Elasto(92),StB,elongation,11.6,...,0.350000,-0.200000,-2.964533,-0.079053,0.384487,2.857143,False,0.866183,0.057465,0.033882
102,26,2022-03-23 16:56:04,cobra_static,2.0,1.99840,2.33,Elasto(95),StA,elongation,11.6,...,0.466347,-0.200000,4.559008,-0.153434,0.492757,2.144327,True,0.747046,0.070126,0.040196


In [58]:
# In DataFrame umwandeln
data_dict_df =  build_data_dict_df(data_dict)

# In Markdown umwandeln und anzeigen
md_text = data_dict_df.to_markdown(tablefmt="github")
display(Markdown(md_text))

|    | Variable                | Kategorie       | Zeichen                        | Deutsch                              | Datentyp   | Einheit   | Beschreibung                                                              |
|----|-------------------------|-----------------|--------------------------------|--------------------------------------|------------|-----------|---------------------------------------------------------------------------|
|  0 | id                      | ptq             | ID                             | ID Messung                           | int64      | -         | Eindeutige ID der Messung                                                 |
|  1 | rope_datetime           | ls3             | $t$                            | Zeitstempel                          | object     | -         | Startzeitpunkt der Messung laut Gerät                                     |
|  2 | treatment               | series          | treatment                      | Behandlung                           | category   | -         | Art der KS: \texttt{free}, \texttt{gefa\_dynamic}, \texttt{cobra\_static} |
|  3 | release_force_target    | series          | $F_{\mathrm{release,target}}$  | Vorspannkraft-Soll                   | float64    | kN        | Geplante Vorspannkraft im Zugseil bei Release                             |
|  4 | rope_release            | ls3             | $F_{\mathrm{release}}$         | Vorspannkraft-Ist                    | float64    | kN        | Tatsächlich realisierte Vorspannkraft im Zugseil bei Release              |
|  5 | cable_max               | ls3             | $F_{\mathrm{cable, max}}$      | Kraftspitze KS                       | object     | kN        | Maximale gemessene Kraftspitze in der KS                                  |
|  6 | sensor_name             | ptq             | sensor                         | Sensorname                           | object     | -         | Bezeichnung des Elastometers                                              |
|  7 | location                | sensor_position | location                       | Position                             | string     | -         | Position des Sensors am Stamm                                             |
|  8 | direction               | sensor_position | direction                      | Richtung                             | string     | -         | Zug- oder Druckseite                                                      |
|  9 | height                  | sensor_position | $h$                            | Höhe                                 | Float64    | m         | Höhe des Sensors am Stamm                                                 |
| 10 | diameter                | sensor_position | $d$                            | Durchmesser                          | Float64    | cm        | Durchmesser des Stammes                                                   |
| 11 | max_strain              | ptq             | $\Delta L_{\mathrm{max}}$      | Dehnung max. gemessen                | float64    | $\mu$m    | Gemessene maximale absolute Randfaserdehnung                              |
| 12 | max_compression         | ptq             | $\Delta L_{\mathrm{comp,max}}$ | Stauchung max. gemessen              | float64    | $\mu$m    | Gemessene maximale absolute Randfaserstauchung                            |
| 13 | m_amplitude             | ptq_osc         | $mA$                           | Manuelle Amplitude                   | float64    | $\mu$m    | Manuell berechnete Amplitude über den Schwingungsabschnitt                |
| 14 | m_amplitude_2           | ptq_osc         | $mA_2$                         | Manuelle Amplitude 2                 | float64    | $\mu$m    | Manuell berechnete Amplitude zwischen 2. Peak und Minimum                 |
| 15 | initial_amplitude       | ptq_osc         | $A$                            | Anfangsamplitude                     | float64    | $\mu$m    | Initiale Amplitude der angepassten Schwingung                             |
| 16 | damping_coeff           | ptq_osc         | $\delta$                       | Dämpfungskoeffizient                 | float64    | 1/s       | Koeffizient der exponentiellen Dämpfung                                   |
| 17 | frequency_damped        | ptq_osc         | $f_{\mathrm{d}}$               | Gedämpfte Frequenz                   | float64    | Hz        | Frequenz der gedämpften Schwingung                                        |
| 18 | phase_angle             | ptq_osc         | $\varphi$                      | Phasenwinkel                         | float64    | rad       | Anfangsphase der Schwingung                                               |
| 19 | y_shift                 | ptq_osc         | $y_0$                          | Vertikaler Versatz                   | float64    | $\mu$m    | Vertikaler Offset der Schwingung                                          |
| 20 | x_shift                 | ptq_osc         | $t_0$                          | Zeitverschiebung                     | float64    | s         | Horizontale Verschiebung der Schwingung                                   |
| 21 | frequency_undamped      | ptq_osc         | $f_0$                          | Ungedämpfte Frequenz                 | float64    | Hz        | Frequenz der ungedämpften Schwingung                                      |
| 22 | damping_ratio           | ptq_osc         | $D$                            | Dämpfungsgrad                        | float64    | -         | Verhältnis von Dämpfung zu Frequenz                                       |
| 23 | metrics_warning         | ptq_osc         | warning                        | Fit-Warnung                          | bool       | -         | Warnung, wenn Qualitätsmetrik Schwellenwerte unterschreitet               |
| 24 | pearson_r               | ptq_osc_metric  | $r$                            | Pearson-Korrelation                  | float64    | -         | Korrelationskoeffizient der Anpassung                                     |
| 25 | nrmse                   | ptq_osc_metric  | $\mathrm{NRMSE}$               | Normalisierter RMSE                  | float64    | -         | Normalisierter mittlerer quadratischer Fehler                             |
| 26 | nmae                    | ptq_osc_metric  | $\mathrm{NMAE}$                | Normalisierter MAE                   | float64    | -         | Normalisierter mittlerer absoluter Fehler                                 |
| 27 | calc_max_strain         | calc_strain     | $\Delta L_{\mathrm{calc,max}}$ | Dehnung max. berechnet               | float64    | $\mu$m    | Berechnete maximale Randfaserdehnung                                      |
| 28 | calc_max_strain_relativ | calc_strain     | $\Delta L_{\mathrm{calc,rel}}$ | Berechnete maximale Randfaserdehnung | float64    | $\mu$m    | Differenz gemessene und berechnete maximale Randfaserdehnung              |
| 29 | strain_difference       | calc_strain     | $\Delta \varepsilon$           | Differenz der Faserdehnung           | float64    | \%        | Relative Differenz der berechneten und gemessenen max. Dehnung            |

### EXPORT: Gefilterte Daten und Datendokumentation exportieren für Weiterverarbeitung (.feather, .csv, .json)

In [59]:
df.to_feather(data_export_directory / "_dataset_clean.feather")
df.to_csv(data_export_directory / "_dataset_clean.csv", sep=";", index=True, encoding="utf-8")

with open(data_export_directory / "_data_dict_clean.json", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=4, ensure_ascii=False)

### LATEX-EXPORT: Datendokumentation als Latex-Tabelle exportieren (.tex)

In [60]:
# Erzeuge DataFrame für LaTeX mit einheitlichem Aufbau
data_dict_df = build_data_dict_df(data_dict, escape_index=True, select_latex_fields=True)

# Exportiere als LaTeX
latex_string = data_dict_df.to_latex(index=False, escape=False)

caption = "Feldversuch 2 - Ergebnisse, Daten Dokumentation"
caption_long = "Feldversuch 2 - Ergebnisse, Daten Dokumentation, Kräfte, Dehnungen und Schwingungsparameter"

save_latex_table(latex_string, caption, latex_export_directory, caption_long=caption_long)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_ergebnisse_daten_dokumentation.tex
